In [128]:
import pandas as pd
from datetime import datetime


In [140]:
game_details = pd.read_csv('nba_games/games_details.csv')
games = pd.read_csv('nba_games/games.csv')

games = games[games['SEASON'] > 2020]


df_home = pd.merge(game_details, games, left_on=["GAME_ID", "TEAM_ID"], right_on=['GAME_ID', "HOME_TEAM_ID"], how="inner")
df_visit = pd.merge(game_details, games, left_on=["GAME_ID", "TEAM_ID"], right_on=['GAME_ID', "VISITOR_TEAM_ID"], how="inner")
# df_combo.head()
# for col in df_home.columns:
#     print(col)

/var/folders/j4/xht4swnx3v38mbt0w75bjqs40000gn/T/ipykernel_61842/971548187.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  game_details = pd.read_csv('nba_games/games_details.csv')


(25635, 49)

In [141]:
# df_home = df_home[df_home['SEASON'] > 2020]
# df_home['GAME_DATE_EST'] = pd.to_datetime(df_home['GAME_DATE_EST'])
df_home.SEASON.min()

2021

Check for nulls and find minimum number of players per team, minimum number of players to start a NBA game is 9

In [144]:
player_teams_df = df_home.groupby(['GAME_ID', 'TEAM_ID']).agg({'PLAYER_ID': ['count']})
print(player_teams_df[player_teams_df.isnull().any(axis=1)])
print(player_teams_df.min())

player_teams_df = df_visit.groupby(['GAME_ID', 'TEAM_ID']).agg({'PLAYER_ID': ['count']})
print(player_teams_df[player_teams_df.isnull().any(axis=1)])
print(player_teams_df.min())


Empty DataFrame
Columns: [(PLAYER_ID, count)]
Index: []
PLAYER_ID  count    10
dtype: int64
Empty DataFrame
Columns: [(PLAYER_ID, count)]
Index: []
PLAYER_ID  count    9
dtype: int64


In [121]:
for idx, row in df_home.iterrows():
    # df_filtered.loc[idx, 'DAY_OF_YEAR'] = row['GAME_DATE_EST'].timetuple().tm_yday
    df_home.loc[idx, 'WEEK_OF_YEAR'] = row['GAME_DATE_EST'].isocalendar().week
    minutes = row['MIN']
    
    if (isinstance(minutes, str)):
        m, s = minutes.split(":")
        df_home.loc[idx, 'SEC'] = int(float(m) * 60 + float(s))
    else :
        df_home.loc[idx, 'SEC'] = 0
    

        


In [122]:
df_home = df_home[['GAME_ID', 'HOME_TEAM_ID', 'PLAYER_ID', 'SEC']]
print(df_home)


        GAME_ID  HOME_TEAM_ID  PLAYER_ID     SEC
14     22200477  1.610613e+09  1630530.0  1667.0
15     22200477  1.610613e+09  1630230.0  1486.0
16     22200477  1.610613e+09   202685.0  1550.0
17     22200477  1.610613e+09  1630529.0  2066.0
18     22200477  1.610613e+09   203468.0  2231.0
...         ...           ...        ...     ...
50990  12100001  1.610613e+09  1630243.0   435.0
50991  12100001  1.610613e+09     2546.0     0.0
50992  12100001  1.610613e+09     2772.0     0.0
50993  12100001  1.610613e+09     2544.0     0.0
50994  12100001  1.610613e+09   201566.0     0.0

[25635 rows x 4 columns]


In [123]:
# df_home.to_csv("player_teams_home.csv")

In [124]:
# player_teams_home_df = df_home.groupby(['GAME_ID', 'HOME_TEAM_ID'])#.agg({'PLAYER_ID': ['count']})
# player_teams_home_df.to_csv("player_teams_home.csv")
gb = df_home.groupby(['GAME_ID', 'HOME_TEAM_ID'])    
game_teams = [gb.get_group(x) for x in gb.groups]


In [125]:
df_home_final = pd.DataFrame()

for x in game_teams:
    tmp = x.nlargest(9, 'SEC')
    num = 1
    for idx, row in tmp.iterrows():
        tmp.loc[idx, 'SEC'] = f"HOME_P{num}"
        num += 1
    tmp.rename(columns={'SEC':'NUM'}, inplace=True)
    tmp = pd.pivot_table(tmp, index=['GAME_ID', 'HOME_TEAM_ID'], columns='NUM', values='PLAYER_ID', aggfunc= lambda r:r)
    df_home_final = pd.concat([df_home_final, tmp])


# tmp1 = game_teams[0].nlargest(9, 'SEC')
# rank = 1
# for idx, row in tmp1.iterrows():
#     tmp1.loc[idx, 'SEC'] = rank
#     rank += 1
# tmp2 = pd.pivot_table(tmp1, index=['GAME_ID', 'HOME_TEAM_ID'], columns='SEC', values='PLAYER_ID', aggfunc= lambda r:r)
# tmp2

In [127]:
for idx, row in df_home_final.iterrows():
    print(row)
    # print(f"GAME: {row['GAME_ID']} | HOME: {row['HOME_TEAM_ID']} | P1: {row['HOME_P1']}")

NUM
HOME_P1    1629659.0
HOME_P2    1629134.0
HOME_P3     203145.0
HOME_P4     201961.0
HOME_P5    1628370.0
HOME_P6    1630559.0
HOME_P7     201599.0
HOME_P8    1630555.0
HOME_P9    1630602.0
Name: (12100001, 1610612747.0), dtype: float64
NUM
HOME_P1    1630567.0
HOME_P2    1630173.0
HOME_P3    1628384.0
HOME_P4    1630625.0
HOME_P5    1627832.0
HOME_P6    1629004.0
HOME_P7    1630551.0
HOME_P8    1629139.0
HOME_P9     201609.0
Name: (12100002, 1610612761.0), dtype: float64
NUM
HOME_P1    1627759.0
HOME_P2    1628369.0
HOME_P3    1626196.0
HOME_P4     203935.0
HOME_P5    1630202.0
HOME_P6    1629057.0
HOME_P7     203471.0
HOME_P8     201143.0
HOME_P9    1627823.0
Name: (12100003, 1610612738.0), dtype: float64
NUM
HOME_P1    1629639.0
HOME_P2    1629130.0
HOME_P3    1629622.0
HOME_P4     202693.0
HOME_P5    1629216.0
HOME_P6    1628997.0
HOME_P7    1629644.0
HOME_P8     203473.0
HOME_P9    1628389.0
Name: (12100004, 1610612748.0), dtype: float64
NUM
HOME_P1    1630162.0
HOME_P2    1629